In [21]:
import plotly.express as px
import pandas as pd
import json
import ast


users_info = pd.read_csv("../data/users.csv", index_col=[0])
users_info["purchase_info"] = users_info.purchase_info.apply(ast.literal_eval)
users_info["increments"] = users_info.increments.apply(ast.literal_eval)
users_info["players"] = users_info.players.apply(ast.literal_eval)
users_info

,name,points,value,purchase_info,increments,players,team_value
0,FC Datalona,0,"€44,700,000","[6630658, 530810, 5200528, 5655110, 5058060, 1...","[1240658, 60810, 1870528, 2425110, 2688060, 25...","[Ter Stegen, Neto, Sergi Roberto, Hugo Mallo, ...",44700000
1,U. B. Conquense,0,"€42,380,000","[1250300, 2910000, 870000, 930000, 880000, 480...","[249700, 1440000, 30000, 190000, 270000, 40000...","[Rubén Blanco, Zaldua, Rubén Duarte, Joan Sast...",42380000
2,Pastafarianism United,0,"€42,230,000","[6820023, 5970000, 1990009, 1310003, 610022, 2...","[2800023, 3380000, 610009, 360003, 250022, 110...","[Cillessen, Yuri, Alberto Moreno, Germán Sánch...",42230000
3,Thomas Party,0,"€40,600,000","[6200000, 1300000, 7510000, 7850000, 3400000, ...","[2870000, 1120000, 3940000, 4480000, 780000, 2...","[Vaclík, Oier Olazábal, Varane, Djené, Pedraza...",40600000
4,Camping Ballena Alegre,0,"€34,060,000","[150100, 5750000, 5560000, 2560000, 660100, 65...","[100, 2710000, 2820000, 1140000, 149900, 25200...","[Sergio Álvarez, Siovas, Damián Suárez, Coke, ...",34060000
5,Santa Bàrbara,0,"€32,020,000","[1180000, 2614990, 483030, 709500, 150000, 105...","[130000, 1454990, 83030, 559500, 0, 2837310, 4...","[Manolo Reina, Ximo Navarro, David López, Davi...",32020000
6,Vetusta FC,0,"€27,070,000","[2099000, 5430000, 3590000, 2670000, 1050000, ...","[589000, 2290000, 1430000, 700000, 40000, 1399...","[Herrerín, Garay, Yeray, Diego Llorente, Picci...",27070000
7,ValeronBalonDeOro,0,"€20,990,000","[1501000, 4100000, 2749000, 276000, 9103000, 6...","[381000, 1110000, 1509000, 4000, 4943000, 3094...","[Rubén, Wass, Rosales, Diakhaby, Kroos, Bardhi...",20990000


In [22]:
def _compute_increments(row):
    rows = []
    for idx, player in enumerate(row.players):
        increment = int(row.increments[idx])
        paid_price = int(row.purchase_info[idx])
        market_value = paid_price-increment
        rows.append({"team": row["name"],
                     "player": player,
                     "increment": increment,
                     "paid_price": paid_price,
                     "market_value": market_value,
                     "relative_increment": ((paid_price/market_value)-1)})
    return pd.DataFrame(rows)


player_transfers = pd.concat(users_info.apply(_compute_increments, axis=1).tolist())
player_transfers.sample(10)

,increment,market_value,paid_price,player,relative_increment,team
0,130000,1050000,1180000,Manolo Reina,0.123810,Santa Bàrbara
7,310880,1710000,2020880,Rubén Pérez,0.181801,FC Datalona
13,1079804,3220000,4299804,Gameiro,0.335343,U. B. Conquense
5,2520005,3980000,6500005,Orellana,0.633167,Camping Ballena Alegre
8,350000,570000,920000,Camarasa,0.614035,U. B. Conquense
0,2800023,4020000,6820023,Cillessen,0.696523,Pastafarianism United
5,110001,150000,260001,Rodrigo Ely,0.733340,Pastafarianism United
16,20000,910000,930000,Nolito,0.021978,Thomas Party
8,10000,430000,440000,Aridane Hernández,0.023256,Thomas Party
6,407440,2380000,2787440,Pere Pons,0.171193,FC Datalona


In [23]:
fig = px.violin(player_transfers.loc[player_transfers.team != "Bayern de Segunda"], y="increment", x="team", points="all", hover_name="player")
fig.update_yaxes(title_text='Paid price - Market value')
fig.update_xaxes(title_text='Team')
fig.show()

In [24]:
fig = px.violin(player_transfers, y="relative_increment", x="team", points="all", hover_name="player")
fig.update_yaxes(title_text='Relative increment', tickformat='%')
fig.update_xaxes(title_text='Team')
fig.show()

In [26]:
players = []
for _, user in users_info.iterrows():
    players.extend(user.players)

json.dumps(players, ensure_ascii=False)

'["Ter Stegen", "Neto", "Sergi Roberto", "Hugo Mallo", "Ferland Mendy", "Raíllo", "Pere Pons", "Rubén Pérez", "Vitolo", "Lucas Vázquez", "Roque Mesa", "Barrenetxea", "Fede Valverde", "Borja Iglesias", "Vinícius", "Dembélé", "Rubén Blanco", "Zaldua", "Rubén Duarte", "Joan Sastre", "Javi López", "Odriozola", "Mathías Olivera", "Portillo", "Camarasa", "Fran Beltrán", "Riqui Puig", "Messi", "Rodrigo", "Gameiro", "Sobrino", "Cillessen", "Yuri", "Alberto Moreno", "Germán Sánchez", "Arbilla", "Rodrigo Ely", "Francis", "Campaña", "Yokuslu", "Szymanowski", "Pione Sisto", "Iago Aspas", "Williams", "Gerard Moreno", "Vaclík", "Oier Olazábal", "Varane", "Djené", "Pedraza", "Felipe Monteiro", "Renan Lodi", "Sidnei", "Aridane Hernández", "Todibo", "James Rodríguez", "Óliver Torres", "Iturraspe", "Portu", "Santi Mina", "Luuk de Jong", "Nolito", "Sergio Álvarez", "Siovas", "Damián Suárez", "Coke", "Mario Gaspar", "Orellana", "Saúl", "Banega", "William Carvalho", "Granero", "Kang-in Lee", "Recio", "Ekam